# 用户和活动关联关系处理


整个数据集中活动数目（events.csv）太多，所以下面的处理我们找出只在训练集和测试集中出现的活动和用户集合，并对他们重新编制索引

In [1]:
#保存数据
import pickle

import itertools

#处理事件字符串
import datetime

import numpy as np
import scipy.io as sio
import scipy.sparse as ss

#相似度/距离
import scipy.spatial.distance as ssd

from collections import defaultdict
from sklearn.preprocessing import normalize

In [2]:
 """
我们只关心train和test中出现的user和event，因此重点处理这部分关联数据

train.csv 有6列：
user：用户ID
event：活动ID
invited：是否被邀请（0/1）
timestamp：ISO-8601 UTC格式时间字符串，表示用户看到该活动的时间
interested, and not_interested

Test.csv 除了没有interested, and not_interested，其余列与train相同
 """
    
# 统计训练集中有多少不同的用户的events
uniqueUsers = set()
uniqueEvents = set()

#倒排表
#统计每个用户参加的活动   / 每个活动参加的用户
eventsForUser = defaultdict(set)
usersForEvent = defaultdict(set)
    
for filename in ["train.csv", "test.csv"]:
    f = open(filename, 'rb')
    
    #忽略第一行（列名字）
    f.readline().strip().decode().split(",")
    
    for line in f:    #对每条记录
        cols = line.strip().decode().split(",")
        uniqueUsers.add(cols[0])   #第一列为用户ID
        uniqueEvents.add(cols[1])   #第二列为活动ID
        
        #eventsForUser[cols[0]].add(cols[1])    #该用户参加了这个活动
        #usersForEvent[cols[1]].add(cols[0])    #该活动被用户参加
    f.close()


n_uniqueUsers = len(uniqueUsers)
n_uniqueEvents = len(uniqueEvents)

print("number of uniqueUsers :%d" % n_uniqueUsers)
print("number of uniqueEvents :%d" % n_uniqueEvents)

#用户关系矩阵表，可用于后续LFM/SVD++处理的输入
#这是一个稀疏矩阵，记录用户对活动感兴趣
userEventScores = ss.dok_matrix((n_uniqueUsers, n_uniqueEvents))
userIndex = dict()
eventIndex = dict()

#重新编码用户索引字典
for i, u in enumerate(uniqueUsers):
    userIndex[u] = i
    
#重新编码活动索引字典    
for i, e in enumerate(uniqueEvents):
    eventIndex[e] = i

n_records = 0
ftrain = open("train.csv", 'rb')
ftrain.readline()
for line in ftrain:
    cols = line.strip().decode().split(",")
    i = userIndex[cols[0]]  #用户
    j = eventIndex[cols[1]] #活动
    
    eventsForUser[i].add(j)    #该用户参加了这个活动
    usersForEvent[j].add(i)    #该活动被用户参加
        
    #userEventScores[i, j] = int(cols[4]) - int(cols[5])   #interested - not_interested
    score = int(cols[4])
    #if score == 0:  #0在稀疏矩阵中表示该元素不存在，因此借用-1表示interested=0
    #userEventScores[i, j] = -1
    #else:
    userEventScores[i, j] = score
    print(str(i) + str(j))
    
ftrain.close()

  
##统计每个用户参加的活动，后续用于将用户朋友参加的活动影响到用户
pickle.dump(eventsForUser, open("PE_eventsForUser.pkl", 'wb'))
##统计活动参加的用户
pickle.dump(usersForEvent, open("PE_usersForEvent.pkl", 'wb'))

#保存用户-活动关系矩阵R，以备后用
sio.mmwrite("PE_userEventScores", userEventScores)


#保存用户索引表
pickle.dump(userIndex, open("PE_userIndex.pkl", 'wb'))
#保存活动索引表
pickle.dump(eventIndex, open("PE_eventIndex.pkl", 'wb'))

    
# 为了防止不必要的计算，我们找出来所有关联的用户 或者 关联的event
# 所谓的关联用户，指的是至少在同一个event上有行为的用户pair
# 关联的event指的是至少同一个user有行为的event pair
uniqueUserPairs = set()
uniqueEventPairs = set()
for event in uniqueEvents:
    i = eventIndex[event]
    users = usersForEvent[i]
    if len(users) > 2:
        uniqueUserPairs.update(itertools.combinations(users, 2))
        
for user in uniqueUsers:
    u = userIndex[user]
    events = eventsForUser[u]
    if len(events) > 2:
        uniqueEventPairs.update(itertools.combinations(events, 2))
 
#保存用户-事件关系对索引表
pickle.dump(uniqueUserPairs, open("FE_uniqueUserPairs.pkl", 'wb'))
pickle.dump(uniqueEventPairs, open("PE_uniqueEventPairs.pkl", 'wb'))

number of uniqueUsers :3391
number of uniqueEvents :13418
14986167
149810127
14984976
14989009
149811701
149811869
11717977
11713368
11715019
117112672
11713198
1171711
11715334
4729163
4726366
4726266
47212166
47212702
4725717
81812214
81811295
81811248
8186138
8182348
8187326
28715191
2871492
287110821
28712063
28712942
287112696
5332863
5338927
53310560
5335015
5335450
53310095
5332201
533915
5333789
5336515
53310615
5338490
5331655
53312768
244312996
244312990
244312224
244312899
24438120
24437082
26256131
26256515
262511626
26254181
26256705
26251371
31759086
31757756
31753690
31755923
317511457
31756679
31754234
18209762
182010342
18204415
182010353
18203791
182012668
1224346
12243792
122412120
12246549
122410372
12248569
7158873
71511121
71512647
71511730
7154358
7153031
323513201
32353099
323511402
32354832
3235732
3235511
180411298
180411701
18044976
180412717
180410096
18048708
18044126
18045870
18046240
180412648
180411869
18049506
18041524
180410668
18046515
18042175
180417

3772645
14157058
14154719
1415569
14158281
141512326
141511086
14631748
14636515
14638713
146311869
146311701
14631524
14634976
14635692
14639346
146310662
14637643
14631673
14638024
146313070
14632217
30765450
30768228
30761495
30763092
30768829
307610729
307612079
30762561
30767126
30768341
30769167
3076563
82012936
82013103
8207203
8207577
8201392
8201242
82077
10043651
10042007
10044195
10048386
10045863
10048739
18835692
18838708
188312741
18832366
18832588
188310885
18835870
18836862
13884601
13881627
138812215
13882500
13885577
13883083
20011016
2004941
2001252
2008469
200160
2008221
200182
2004921
20012657
20010571
2002763
2004916
2004258
2008186
2007779
2005825
2009622
2009942
2008476
200541
287813103
28785705
287812831
287810609
28787577
28781242
136213103
136210814
13627605
136210609
13627203
13622912
32393774
32396728
32393979
3239104
32399789
32399119
2024130
20246195
202410925
202410309
20241556
202413179
18311139
1835221
1832598
18311982
1832892
1832531
32047327
32046299

16643136
16646878
16649767
1664205
166410687
166413065
166410294
655912
652289
655450
659588
654976
653753
657654
7344371
734764
7345839
73413013
7341149
734318
164012812
1640298
164010474
16405758
16406330
16408703
16405537
16409204
16404687
16405726
16405890
16408151
1640147
16407365
9314917
9316914
9314107
9314037
93112477
9319415
25737685
257311978
25735238
25736975
25733633
25733447
1887699
1888999
18810036
18810111
18812748
18813027
2397002
2393443
2399983
2396515
23911124
2392514
23921
29903238
29901869
29901869
29903238
29902097
299011377
29908094
29905932
299013246
29908944
29902097
29908368
29902132
29903628
29908094
29901869
29907353
29905582
29903352
299012960
27503641
27507187
27504354
27506992
275012480
27508349
2750547
2750781
27503249
30964464
3096561
30966023
30969890
30961449
309612561
6475916
6471556
6471403
64710309
64710925
64713014
64710011
134010769
134011698
1340375
13407178
134011516
13405462
1340704
227112230
22716028
227111227
22717547
22717724
227112426
2271

19406366
19405717
19406266
19404798
27092637
270911124
270913056
270911240
27093356
27095692
27096845
27091378
27096515
108811366
10881057
108810672
108810932
10888139
21928257
21927203
21928621
21923365
21926697
219213228
20452289
20458829
20457413
20452589
20456515
20452782
20454976
20455450
20457654
4825545
4829126
4828593
4823556
482587
4823346
174810609
17487605
174812936
17487203
17482912
17487577
174813103
22452201
224512401
22455672
22456577
22455301
22454702
22456845
22457189
203512012
2035288
203511385
20351674
20356976
20359634
20355401
20351412
31277330
31276732
31272906
31271865
31274237
31277558
24453086
244511190
24454205
24455148
24452494
24454882
24451873
244511365
24455252
24457393
24454613
24453874
279713300
27977024
27974634
279710019
27974789
2797861
27978428
279711446
10822514
10826834
1082753
10824683
10823753
1082536
13826292
13821786
13826600
138210447
1382885
13827002
13826659
17204300
17204572
17203940
17206191
17203089
172010751
301810925
30187082
30184757
3

16885529
16882816
16883331
17214976
17219506
17218454
17215870
17216515
17216862
28312873
28311410
2831728
2832440
28310762
2839545
18569307
185612431
185611059
18568381
33635991
336311567
33632454
33639806
33633045
33633045
33639806
33639380
336312161
336311567
33639380
336312161
1637106
16379793
1637180
16371054
16379151
16374057
283312305
28334285
28334510
28332566
28333897
28337067
28454976
28455692
28456515
284511874
284510761
28452800
330313103
33033870
330377
33034656
33033668
33033780
17544656
175410609
17542645
1754834
17549730
175413103
175412936
141712036
14172632
14176277
14174872
14179072
141712289
17437921
17437831
17433712
174312996
17436850
281311869
28136732
28139122
28133426
28136177
28137185
28136515
28139469
138713103
13871242
138713228
13877203
13877605
13874244
13873668
31662963
31668316
31666853
316610024
31668749
31663624
31661135
31661686
30894821
30894590
30894290
30897403
308912783
308912708
308910173
30898608
30897907
302518
302414
305377
303145
3012046
3023

...

11846515
11844976
326310128
326311012
32632702
32633370
32631633
32632972
247312546
24736546
247310596
247312934
24734513
24731792
23210776
2327081
23210378
2325690
2329763
2326313
2323017
14063254
14068247
14064204
14067563
14069752
14069934
8484202
84813308
8483699
8488549
8487906
8489437
8488901
8489434
13804394
13802530
13802186
13803646
13804517
13803435
13809823
138013071
13808962
138013047
13805315
9452711
9457205
9451660
94512106
9459852
94510039
9456464
9452503
94510853
945690
945859
9453493
945459
945799
94511981
9458274
9451776
94511225
9457824
9457294
16841524
16846515
16847739
16842124
168411701
16844976
168411869
29008211
29007740
29004745
29005830
290011775
29001182
31981938
31985818
31988600
31981202
31989148
319812872
31982913
32882782
328812009
32888136
32883101
32887443
32886723
30227037
3022202
30229372
30226515
30224976
302211874
30229485
32003233
32007756
320011366
320011102
32009705
320012238
3894694
3897869
38913134
389125
38912228
38912066
31487307
31484669
314

26573882
2657130
26571115
2657288
265710000
5304656
5307203
53012936
530335
53013103
5307605
152311869
15231524
15234976
15235243
15236515
152311701
25646785
2564223
256412194
25648394
256411195
256412447
25648219
25646919
25646579
25647561
2564502
2564558
57412858
5744254
5748920
5742956
5748162
111511652
111513155
11154201
11159800
111555
111511470
111512787
1115184
160767
1602332
1605107
1609762
1607431
16012581
1604890
1602332
16010597
1605107
1602305
1609762
23625387
23626862
23622324
23626515
23628024
23621511
23629506
16064656
1606834
16067203
160613103
16064244
16061242
27132912
27137795
2713741
27137203
271313228
27137605
23726341
23729984
23728839
2372659
2372359
23724241
29865992
29863368
29865831
298612226
29868694
29865019
2986876
298610716
526711
523004
522971
529185
5211550
522828
522964
523926
523876
5212408
5210332
529043
526808
1826715
18211458
18211793
18210985
1822831
1824667
5524787
5523824
55213297
5528751
5529721
5526402
118011748
11805254
11803295
11808406
11805

12021641
12028914
12027106
12022088
1202885
120210723
70086
70011124
7002514
7009371
7007328
7006515
7005692
7002197
4155450
4155672
4159645
4156515
4152782
415105
111911683
11195530
11197090
11191681
11199022
111910789
11193312
1119280
111910669
18947939
18945312
18945312
18947096
18947497
189410011
18943954
189413161
1894619
189410769
18947117
18949738
18947972
18944781
18948543
18941146
189412783
18949766
189411797
18946977
189411797
18941146
189411447
18947939
189410672
146611133
14665672
14666884
146611342
14668487
14667885
12251907
12255312
122512783
12253736
12253220
12256977
12254821
1681221
1686166
168262
1683609
1683601
1681584
13921334
13921778
13929608
13929738
19013308
1907910
19011574
1909434
1905621
1906694
16512424
1651927
1652818
1659967
16512626
165869
1655742
1652993
11110611
1116640
1117809
1117344
11110778
1111103
2188694
2185121
2183368
21810715
218226
2186323
31913103
31912936
3197203
3197605
3195168
31910814
3194656
268410910
26842852
26845410
268411087
26841210

29564026
2956728
295612160
29568153
29562644
295612324
295613104
29569630
29566039
29562295
29565704
29565120
29562644
29562345
29561924
29568204
2956271
29569061
14408822
144012031
14409074
14403062
144010366
14407219
8215923
8215672
8217501
8215246
8212166
8211057
82110392
82110663
821880
8219896
8213233
82178
8215813
8219266
82111102
31789707
31786334
31787050
31787257
31789105
317812019
31781176
3178588
31788053
31781706
31785187
31781744
317811368
31781163
31781813
317812167
31789637
31782387
31789105
317813272
3178230
31782387
31781744
317811368
31781163
31785360
31784210
31785691
317812071
31785371
31782460
31784662
31785875
31789010
31784664
22316511
22313748
22316185
223110927
22315734
2231119
22313270
22319212
223112914
22316192
22314145
22318762
22312683
2231240
22311038
223112974
22314075
223111530
22315181
22314229
22317176
223111025
22312516
22314280
223112885
22318725
22315283
223112330
22314865
223110172
22317786
22317564
22319212
223110547
22314280
22316511
22314066
22

32553455
325510055
325558
32554363
32557500
32551377
325513374
3255299
325511736
325510532
325511636
32553813
32557702
32558601
32552592
32556184
3255293
32556904
325511718
325511118
325511323
32558630
32557342
325513032
32554927
32556779
32558590
325511364
32555871
31871452
318712169
318710800
318712721
31873159
318711598
31876623
19376515
19374366
193710511
193712141
19374976
19375923
19372092
2291237
22918403
2291908
22916754
2291806
22913185
22911400
22918173
229111987
22915664
216964
2162587
2166682
21612252
216266
2163723
2162613
216724
2332262
23312801
2334564
2335540
23311552
2339771
24305587
24307508
24308493
24307333
24307404
24304977
29554270
29557462
295511199
29555949
29551824
295510887
18456838
18452395
184511105
184510662
18451774
18458454
296410446
29649747
29643500
296410131
29648092
29644376
29649721
296411318
29649990
29648755
2964130
296411017
29642443
135413255
13548080
1354353
13543368
135412226
13545569
18115110
18116613
18115692
18116515
18116523
181111632
25931

In [3]:
#训练集和测试集中出现的用户数目和事件数目远小于users.csv出现的用户数和events.csv出现的事件数